In [1]:
# importing libraries
import nltk
import random 
from nltk.corpus import movie_reviews 
import csv
import numpy as np

import re
from nltk.corpus import stopwords
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
import pickle

In [2]:
#listing positive review files and negative review files
positive_review_files = [] # only positive review files
negative_review_files = [] # only negative review files

In [3]:
fileids = movie_reviews.fileids()
for i in fileids:
    filename = str(i)
    filename = filename.split('/')
    if filename[0] == 'neg':
        negative_review_files.append(str(i))
    elif filename[0] == 'pos':
        positive_review_files.append(str(i))

In [4]:
# listing all te positive sentences from the files
list_of_positive_sentences = [] # list of positive sentences
# listing all the negative sentences from the files 
list_of_negative_sentences = [] # list of negative sentences

In [5]:
for i in positive_review_files:
    words = movie_reviews.raw(i)
    sentences = words.split('\n')
    for j in sentences:
        list_of_positive_sentences.append(j)
        

In [6]:
for i in negative_review_files:
    words = movie_reviews.raw(i)
    sentences = words.split('\n')
    for j in sentences:
        list_of_negative_sentences.append(j)
    

In [7]:
list_of_positive_sentences[:2]

[u"films adapted from comic books have had plenty of success , whether they're about superheroes ( batman , superman , spawn ) , or geared toward kids ( casper ) or the arthouse crowd ( ghost world ) , but there's never really been a comic book like from hell before . ",
 u"for starters , it was created by alan moore ( and eddie campbell ) , who brought the medium to a whole new level in the mid '80s with a 12-part series called the watchmen . "]

In [ ]:
# this is another source of data that is appended to the list of positive and negative sentences accordingly.
with open('sentimenter.csv') as f:
    r = csv.reader(f)
    for row in r:
        if row[1] == '0':
            list_of_negative_sentences.append(row[3])
        elif row[1] == '1':
            list_of_positive_sentences.append(row[3])
            

In [ ]:
with open('large/sentimentasset.csv') as f:
    r = csv.reader(f)
    for row in r:
        if row[1] == '0':
            list_of_negative_sentences.append(row[3])
        elif row[3] == '1':
            list_of_positive_sentences.append(row[3])

In [ ]:
X_train = []
y_train = []

In [ ]:
# putting words in the training set
for i in list_of_negative_sentences:
    X_train.append(i)
    y_train.append('negative')
for i in list_of_positive_sentences:
    X_train.append(i)
    y_train.append('positive')

In [ ]:
len(X_train)

In [ ]:
STOPWORDS = set(stopwords.words('english'))
list1 = []
for i in STOPWORDS:
    list1.append(i)
STOPWORDS = list1
list1 = []
for i in range(len(STOPWORDS)):
    if len(STOPWORDS[i]) < 2:
        list1.append(STOPWORDS[i])
STOPWORDS = list1

In [ ]:
# Its time to cleanse the data
REPLACE_BY_SPACE_RE = re.compile(r'[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile(r'[^0-9a-z #+_]')
def text_prepare(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower()# lowercase text
    text = re.sub(REPLACE_BY_SPACE_RE," ",text)# replace REPLACE_BY_SPACE_RE symbols by space in text
    text = re.sub(BAD_SYMBOLS_RE,"",text)# delete symbols which are in BAD_SYMBOLS_RE from text
    temp = re.split(" ",text)
    for i in temp:
        if i == "":
            temp.remove(i)
    a = ''
    for i in temp:
        if i not in STOPWORDS:
            a = a + i + ' '
    a = a[:len(a)-1]
    return a

In [ ]:
# in this block we have stored the cleansed data 
X_train = [text_prepare(x) for x in X_train]

In [ ]:
with open('train.csv','a+') as f:
    w = csv.writer(f)
    with open('label.csv','a+') as g:
        v = csv.writer(g)
        while len(x_train) != 0:
            temp = random.choice(x_train)
            idx = x_train.index(temp)
            w.writerow([temp])
            v.writerow([Y_train[idx]])
            x_train.remove(temp)
            Y_train.remove(Y_train[idx])
            if len(x_train) % 100 ==0:
                print len(x_train)
        

# You can start creating your model from the saved data
we will create a `positive-negative.pkl` as our model 

In [7]:
X_train = []
y_train = []
with open('train.csv','r') as f:
    w = csv.reader(f)
    for i in w:
        X_train.append(i)
with open('label.csv','r') as f:
    r = csv.reader(f)
    for i in r:
        y_train.append(i)

In [8]:
temp = X_train
temper = y_train
X_train = []
y_train = []
for i in temp:
    X_train.append(i[0])
for j in temper: 
    y_train.append(j[0])
    

In [9]:
X_train[:3]

['exams comin up',
 'sweetangel69 doin good',
 'sydneyfamous dont forget to tell your friends about the contest so they can get in for free']

### words count and reverse dictionary
Here we are going to count the words  and assign the dictionaries to count them.

In [10]:
# Dictionary of all tags from train corpus with their counts.
tags_counts = {}

labels = []
for i in y_train:
    labels.append(i)
tags_counts = dict(Counter(labels))

# Dictionary of all words from train corpus with their counts.
words_counts = {}

words = []
for i in X_train:
    for word in re.split(" ",i):
        words.append(word)
words_counts = dict(Counter(words))
for i in words_counts.keys():
    if i == "":
        words_counts.pop(i)

In [11]:
# most common words 
most_common_words = sorted(words_counts.items(), key=lambda x: x[1], reverse=True)[:20]
print most_common_words

[('to', 117532), ('the', 111398), ('my', 65442), ('and', 63340), ('is', 50605), ('you', 47841), ('it', 47162), ('in', 45678), ('for', 42254), ('of', 40322), ('im', 36563), ('on', 33635), ('me', 32345), ('so', 31108), ('have', 30166), ('that', 29917), ('but', 28279), ('just', 24555), ('not', 24018), ('at', 23374)]


## Using the tf-idf vectorizer
this is for the vectorising the words that we get.
we will get a `positive-negative-tfidf` as a result of this. 

In [12]:
def tfidf_features(X_train):
    
    
    tfidf_vectorizer = TfidfVectorizer(min_df=5, max_df=0.9, ngram_range=(1,2))####### YOUR CODE HERE #######
    tfidf_vectorizer = tfidf_vectorizer.fit(X_train)
    X_train = tfidf_vectorizer.transform(X_train)
    
    return tfidf_vectorizer,X_train, tfidf_vectorizer.vocabulary_

In [13]:
tfidf_vectorizer, X_train_tfidf, tfidf_vocab = tfidf_features(X_train)
tfidf_reversed_vocab = {i:word for word,i in tfidf_vocab.items()}
#basically tfidf_reversed_vocab is a dictionary that stores the indexes to the vocabs created by the TfidfVectorizer
# I learned it from the video tutorials

In [ ]:
pickle.dump(tfidf_vectorizer,open('positive-negative-tfidf.pkl', 'wb'))

In [14]:
temp = []
for i in y_train:
    ls = []
    ls.append(i)
    temp.append(ls)
y_train = temp

In [15]:
mlb = MultiLabelBinarizer(classes=sorted(tags_counts.keys()))
y_train = mlb.fit_transform(y_train)

In [16]:
pickle.dump(mlb,open('multilabel-binarizer.pkl','wb'))

## training starts here

In [ ]:
def train_classifier(X_train, y_train):
    """
      X_train, y_train — training data
      
      return: trained classifier
    """
    
    #Create and fit LogisticRegression wraped into OneVsRestClassifier.
    classifier = OneVsRestClassifier(LogisticRegression(C=1.0))
    classifier = classifier.fit(X_train,y_train)
    return classifier

In [ ]:
tag_classifier = train_classifier(X_train_tfidf, y_train)

In [ ]:
X_train_tfidf.shape

In [ ]:
pickle.dump(tag_classifier, open('positive-negative.pkl', 'wb'))

# Now testing the model we have

In [18]:
def generate_output(tfidf_vectorizer,tag_classifier,mlb,test):
    test1 = []
    test1.append(test)
    x_test_tfidf = tfidf_vectorizer.transform(test1)
    test_predictions = tag_classifier.predict(x_test_tfidf)
    test_pred_inversed = mlb.inverse_transform(test_predictions)
    return test_pred_inversed

In [4]:
def unpickle_file(filename):
    """Returns the result of unpickling the file content."""
    with open(filename, 'rb') as f:
        return pickle.load(f)

In [17]:
tfidf_vectorizer = unpickle_file('positive-negative-tfidf.pkl')
tag_classifier = unpickle_file('positive-negative.pkl')
multilabel_binarizer = unpickle_file('multilabel-binarizer.pkl')

In [19]:
testing_data = raw_input('enter a data >> ')
print generate_output(tfidf_vectorizer,tag_classifier,multilabel_binarizer,testing_data)

enter a data >> i am mad
[('negative',)]
